In [1]:
import pandas as pd
import numpy as np
import glob
import datetime

In [2]:
zip_code=['20001','20002','20003','20004','20005','20006','20007','20008','20009','20010','20011','20012',
         '20015','20016','20017','20018','20019','20020','20024','20032','20036','20037']

In [3]:
path_all=r'../Home Sales/10 yr'
files=glob.glob(path_all+"/*")
frame=pd.DataFrame()
list_=[]
for file in files:
    df = pd.read_csv(file,index_col=None, header=0,dtype={'Zip Code':str})
    df["Current Price"]=df["Current Price"].astype(str)
    df["Current Price"]=df["Current Price"].map(lambda x: str.split(x,"$")[-1].replace(",",""))
    df["Current Price"]=df["Current Price"].astype(float)
    list_.append(df)
frame=pd.concat(list_)


C:\Users\19018\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [4]:
frame["year"]=frame["Status Contractual Search Date"].map(lambda x: str.split(x,"/")[-1])

In [5]:
frame["year"]="20"+frame["year"]

In [6]:
frame["year"]=frame["year"].map(lambda x: int(x))

In [7]:
frame["Structure Type"]=frame["Structure Type"].replace("Row/Townhouse",
                                                                       "Townhouse").replace("End of Row/Townhouse",
                                                                                           "Townhouse").replace("Unit/Flat/Apartment",
                                                                                                               "Apartment").replace("Penthouse Unit/Flat/Apartment",
                                                                                                                                   "Apartment")

In [8]:
frame["Structure Type"].unique()

array(['Townhouse', 'Twin/Semi-Detached', 'Apartment', 'Other', nan,
       'Detached', 'Garage/Parking Space', 'Mobile Pre 1976',
       'Manufactured'], dtype=object)

In [9]:
df=frame.groupby(["Zip Code","year","Structure Type"])["Current Price"].mean()

In [10]:
df=df.reset_index(["Zip Code","year","Structure Type"])

In [11]:
df.to_csv("home sales by year and structure.csv")

In [12]:
years=np.arange(2009,2020,1)

In [13]:
types=df["Structure Type"].unique()
for i in range(len(types)):
    df_sub=df[df["Structure Type"]==types[i]].reset_index()
    a=pd.DataFrame(zip_code,columns={"Zip Code"})
    for j in range(len(years)):
        b=df_sub[df_sub["year"]==years[j]][["Zip Code","Current Price"]]
        b=b.rename(columns={"Current Price":years[j]})
        a=a.merge(b,on="Zip Code",how="left")
    types[i]=types[i].replace("/","_")
    name="home sales by structure/home sales by years of "+types[i]+".csv"
    a.to_csv(name,index=False)

In [32]:
dates=pd.date_range(start='20081231',end='20200101',freq="BA-AUG")
dates=dates.date

In [33]:
comp=pd.DataFrame(dates,columns={"Date"})

In [34]:
comp["Date"]=comp["Date"].map(lambda x: str(x))

In [35]:
sp=pd.read_csv("GSPC.csv")

In [36]:
data=comp.merge(sp[["Date","Adj Close"]],on="Date")

In [37]:
data=data.rename(columns={'Adj Close':"SP500"})

In [38]:
data["SP return"]=(data["SP500"]-data["SP500"].shift(1))/data["SP500"].shift(1)

In [39]:
data=data.drop(columns={"SP500"})

In [40]:
iyr=pd.read_csv("IYR.csv")

In [41]:
data=data.merge(iyr[["Date","Adj Close"]],on="Date")

In [42]:
data=data.rename(columns={'Adj Close':"IYR"})

In [43]:
data["IYR return"]=(data["IYR"]-data["IYR"].shift(1))/data["IYR"].shift(1)

In [26]:
data=data.drop(columns={"IYR"})

In [27]:
data.to_csv("Comparison.csv")

In [44]:
data

,Date,SP500,SP return,IYR,IYR return
0,2009-08-31,1020.619995,NaN,27.611008,NaN
1,2010-08-31,1049.329956,0.028130,36.030350,0.304927
2,2011-08-31,1218.890015,0.161589,41.971088,0.164881
3,2012-08-31,1406.579956,0.153984,50.034519,0.192119
4,2013-08-30,1632.969971,0.160951,49.289959,-0.014881
5,2014-08-29,2003.369995,0.226826,61.067677,0.238948
6,2015-08-31,1972.180054,-0.015569,60.163338,-0.014809
7,2016-08-31,2170.949951,0.100787,73.550682,0.222517
8,2017-08-31,2471.649902,0.138511,75.367607,0.024703
9,2018-08-31,2901.520020,0.173920,80.115425,0.062995


,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-08-03,990.219971,1003.609985,990.219971,1002.630005,1002.630005,5603440000
1,2009-08-04,1001.409973,1007.119995,996.679993,1005.650024,1005.650024,5713700000
2,2009-08-05,1005.409973,1006.640015,994.309998,1002.719971,1002.719971,7242120000
3,2009-08-06,1004.059998,1008.000000,992.489990,997.080017,997.080017,6753380000
4,2009-08-07,999.830017,1018.000000,999.830017,1010.479980,1010.479980,6827090000
...,...,...,...,...,...,...,...
2574,2019-10-23,2994.010010,3004.780029,2991.209961,3004.520020,3004.520020,3392870000
2575,2019-10-24,3014.780029,3016.070068,3000.419922,3010.290039,3010.290039,3692600000
2576,2019-10-25,3003.320068,3027.389893,3001.939941,3022.550049,3022.550049,3370370000
2577,2019-10-28,3032.120117,3044.080078,3032.120117,3039.419922,3039.419922,3521230000


In [29]:
# years=np.arange(2009,2020,1)
# for i in range(len(files)):
#     path=allfiles[i]
#     allFiles = glob.glob(path + "/*.csv")
#     frame=pd.DataFrame()
#     list_=[]
#     for file in allFiles:
#         df = pd.read_csv(file,index_col=None, header=0,dtype={'Zip':str})
#         df["Current Price"]=df["Current Price"].astype(str)
#         df["Current Price"]=df["Current Price"].map(lambda x: str.split(x,"$")[-1].replace(",",""))
#         df["Current Price"]=df["Current Price"].astype(float)
#         df=df.rename(columns={"Current Price":years[i]})
#         list_.append(df)
#     frame=pd.concat(list_)
#     ave=pd.DataFrame(frame.groupby(by=["Zip","Structure Type"])[years[i]].mean())
#     ave=ave.reset_index(["Zip","Structure Type"])
#     ave[years[i]]=round(ave[years[i]])
#     if i==0:
#         data=ave
#     else:
#         data=data.merge(ave,how="outer")

# data.to_csv("home sales by structure/home sales by year and structure.csv")

In [30]:
# dfsale=pd.DataFrame(data=zip_code,columns={"Zip"},dtype=str)
# dfapp=pd.DataFrame(data=zip_code,columns={"Zip"},dtype=str)
# types=data["Structure Type"].unique()
# for i in range(len(types)):
#     data2=data[data["Structure Type"]==types[i]].reset_index(drop=True)
#     data2=data2.drop(columns="Structure Type")
#     types[i]=types[i].replace("/","_")
#     name="home sales by structure/home sales by years of "+types[i]+".csv"
#     data2.to_csv(name,index=False)